# ONE

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pgmpy.models import DiscreteBayesianNetwork  # Perubahan di sini
from pgmpy.estimators import MaximumLikelihoodEstimator
from pgmpy.inference import VariableElimination
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB
import ipywidgets as widgets
from IPython.display import display
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# %% [markdown]
# # Prediksi Performa Akademik Mahasiswa
# ## Menggunakan Jaringan Bayesian Diskrit

# %% [markdown]
# ## 1. Pemrosesan Data Awal

# %%
# Membaca dataset
try:
    data = pd.read_csv('./../Student_performance_data_.csv')
    print("Data berhasil dimuat. Contoh data:")
    display(data.head())
except FileNotFoundError:
    print("File dataset.csv tidak ditemukan. Pastikan file ada di direktori yang benar.")

# Pemetaan nilai untuk interpretasi
pemetaan = {
    'Gender': {0: 'Laki-laki', 1: 'Perempuan'},
    'Ethnicity': {0: 'Kaukasia', 1: 'Afrika-Amerika', 2: 'Asia', 3: 'Lainnya'},
    'ParentalEducation': {
        0: 'Tidak Sekolah', 
        1: 'SMA', 
        2: 'Perguruan Tinggi (Sebagian)',
        3: 'Sarjana',
        4: 'Pascasarjana'
    },
    'Tutoring': {0: 'Tidak', 1: 'Ya'},
    'GradeClass': {0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'F'}
}

# %% [markdown]
# ## 2. Diskritisasi Data

# %%
def diskritisasi_data(df):
    # Membuat kategori untuk variabel kontinu
    df['StudyTime_Cat'] = pd.cut(df['StudyTimeWeekly'],
                                bins=[0, 5, 10, 15, 20, np.inf],
                                labels=['Sangat Sedikit', 'Sedikit', 'Sedang', 'Banyak', 'Sangat Banyak'])
    
    df['Absences_Cat'] = pd.cut(df['Absences'],
                               bins=[0, 3, 6, 10, 15, np.inf],
                               labels=['Sangat Rendah', 'Rendah', 'Sedang', 'Tinggi', 'Sangat Tinggi'])
    
    df['GPA_Cat'] = pd.cut(df['GPA'],
                          bins=[0, 2.0, 2.5, 3.0, 3.5, 4.0],
                          labels=['F', 'D', 'C', 'B', 'A'])
    
    return df

data = diskritisasi_data(data)

# %% [markdown]
# ## 3. Pembangunan Model Jaringan Bayesian

# %%
# Mendefinisikan struktur jaringan
edges = [
    ('ParentalEducation', 'ParentalSupport'),
    ('ParentalSupport', 'StudyTime_Cat'),
    ('StudyTime_Cat', 'GPA_Cat'),
    ('Absences_Cat', 'GPA_Cat'),
    ('Tutoring', 'GPA_Cat'),
    ('GPA_Cat', 'GradeClass')
]

model = DiscreteBayesianNetwork(edges)  # Perubahan di sini

# Membagi data latih dan uji
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Melatih model
model.fit(train_data, estimator=MaximumLikelihoodEstimator)

# %% [markdown]
# ## 4. Evaluasi Model

# %%
# Fungsi untuk evaluasi
def evaluasi_model(data_uji, model):
    inferensi = VariableElimination(model)
    prediksi = []
    
    for _, row in data_uji.iterrows():
        evidence = {
            'StudyTime_Cat': row['StudyTime_Cat'],
            'Absences_Cat': row['Absences_Cat'],
            'Tutoring': row['Tutoring']
        }
        hasil = inferensi.map_query(['GradeClass'], evidence=evidence)
        prediksi.append(hasil['GradeClass'])
    
    akurasi = accuracy_score(data_uji['GradeClass'], prediksi)
    print(f"Akurasi Prediksi: {akurasi:.2f}")
    
    # Confusion matrix
    print("\nMatriks Konfusi:")
    print(pd.crosstab(data_uji['GradeClass'], prediksi,
                     rownames=['Aktual'], colnames=['Prediksi']))

evaluasi_model(test_data, model)

# %% [markdown]
# ## 5. Antarmuka Interaktif

# %%
# Fungsi prediksi interaktif
def prediksi_interaktif(waktu_belajar, absensi, bimbingan):
    # Konversi input ke kategori
    kategori_studi = pd.cut([waktu_belajar], [0,5,10,15,20,np.inf],
                          labels=['Sangat Sedikit','Sedikit','Sedang','Banyak','Sangat Banyak'])[0]
    kategori_absen = pd.cut([absensi], [0,3,6,10,15,np.inf],
                          labels=['Sangat Rendah','Rendah','Sedang','Tinggi','Sangat Tinggi'])[0]
    
    # Melakukan prediksi
    inferensi = VariableElimination(model)
    hasil = inferensi.query(
        ['GradeClass'],
        evidence={
            'StudyTime_Cat': kategori_studi,
            'Absences_Cat': kategori_absen,
            'Tutoring': bimbingan
        }
    )
    
    # Visualisasi hasil
    plt.figure(figsize=(10,5))
    sns.barplot(x=list(pemetaan['GradeClass'].values()), y=hasil.values)
    plt.title('Probabilitas Prediksi Nilai')
    plt.ylabel('Probabilitas')
    plt.xlabel('Grade')
    plt.show()
    
    # Menampilkan hasil utama
    grade_prediksi = hasil.state_names['GradeClass'][np.argmax(hasil.values)]
    print(f"\nPrediksi Grade: {pemetaan['GradeClass'][grade_prediksi]}")
    print(f"Dengan probabilitas: {np.max(hasil.values):.2f}")

# Membuat widget interaktif
waktu_slider = widgets.IntSlider(min=0, max=40, value=10, description='Waktu Belajar:')
absen_slider = widgets.IntSlider(min=0, max=30, value=5, description='Absensi:')
bimbingan_dropdown = widgets.Dropdown(
    options=[('Tidak', 0), ('Ya', 1)],
    description='Bimbingan Belajar:'
)

widgets.interactive(
    prediksi_interaktif,
    waktu_belajar=waktu_slider,
    absensi=absen_slider,
    bimbingan=bimbingan_dropdown
)

# %% [markdown]
# ## 6. Analisis Sensitivitas

# %%
# Fungsi analisis sensitivitas
def analisis_sensitivitas():
    inferensi = VariableElimination(model)
    faktor = ['StudyTime_Cat', 'Absences_Cat', 'Tutoring']
    
    print("Analisis Pengaruh Faktor terhadap Grade A:")
    for faktor in faktor:
        print(f"\nFaktor: {faktor}")
        for nilai in data[faktor].unique():
            prob = inferensi.query(['GradeClass'], evidence={faktor: nilai})
            prob_a = prob.values[0]  # Probabilitas Grade A
            print(f"{nilai}: {prob_a:.2f}")

analisis_sensitivitas()

# TWO

In [1]:
# %%
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pgmpy.models import DiscreteBayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.estimators import MaximumLikelihoodEstimator
from pgmpy.inference import VariableElimination
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import ipywidgets as widgets
from IPython.display import display
import warnings
warnings.filterwarnings('ignore')

In [2]:


# %% [markdown]
# ## 1. Pemrosesan Data yang Lebih Robust

# %%
# Membaca dataset dengan penanganan error
try:
    data = pd.read_csv('./../Student_performance_data_.csv')
    print("Data berhasil dimuat. Contoh 5 data pertama:")
    display(data.head())
except Exception as e:
    print(f"Error saat memuat data: {e}")

# Pastikan kolom yang diperlukan ada
required_columns = ['StudyTimeWeekly', 'Absences', 'Tutoring', 'GradeClass']
if not all(col in data.columns for col in required_columns):
    missing = [col for col in required_columns if col not in data.columns]
    print(f"Kolom yang hilang: {missing}")

# %% [markdown]
# ## 2. Diskritisasi Data yang Diperbaiki

# %%
def diskritisasi_data(df):
    """Fungsi untuk mengkonversi variabel kontinu ke kategori diskrit"""
    try:
        # Pastikan nilai tidak null
        df = df.copy().dropna(subset=['StudyTimeWeekly', 'Absences', 'GPA'])
        
        # Kategorisasi waktu belajar
        df['StudyTime_Cat'] = pd.cut(
            df['StudyTimeWeekly'],
            bins=[0, 5, 10, 15, 20, np.inf],
            labels=['0-5', '5-10', '10-15', '15-20', '20+'],
            right=False
        )
        
        # Kategorisasi absensi
        df['Absences_Cat'] = pd.cut(
            df['Absences'],
            bins=[0, 3, 6, 10, 15, np.inf],
            labels=['0-3', '4-6', '7-10', '11-15', '16+'],
            right=False
        )
        
        # Kategorisasi GPA sesuai GradeClass
        df['GPA_Cat'] = pd.cut(
            df['GPA'],
            bins=[0, 2.0, 2.5, 3.0, 3.5, 4.0],
            labels=['F', 'D', 'C', 'B', 'A'],
            right=False
        )
        
        # Konversi ke tipe kategori
        for col in ['StudyTime_Cat', 'Absences_Cat', 'GPA_Cat']:
            df[col] = df[col].astype('category')
            
        return df
        
    except Exception as e:
        print(f"Error dalam diskritisasi: {e}")
        return None

data_diskrit = diskritisasi_data(data)
if data_diskrit is not None:
    print("\nData setelah diskritisasi:")
    display(data_diskrit[['StudyTimeWeekly', 'StudyTime_Cat', 'Absences', 'Absences_Cat', 'GPA', 'GPA_Cat']].head())

Data berhasil dimuat. Contoh 5 data pertama:


,StudentID,Age,Gender,Ethnicity,ParentalEducation,StudyTimeWeekly,Absences,Tutoring,ParentalSupport,Extracurricular,Sports,Music,Volunteering,GPA,GradeClass
0,1001,17,1,0,2,19.833723,7,1,2,0,0,1,0,2.929196,2.0
1,1002,18,0,0,1,15.408756,0,0,1,0,0,0,0,3.042915,1.0
2,1003,15,0,2,3,4.210570,26,0,2,0,0,0,0,0.112602,4.0
3,1004,17,1,0,3,10.028829,14,0,3,1,0,0,0,2.054218,3.0
4,1005,17,1,0,2,4.672495,17,1,3,0,0,0,0,1.288061,4.0



Data setelah diskritisasi:


,StudyTimeWeekly,StudyTime_Cat,Absences,Absences_Cat,GPA,GPA_Cat
0,19.833723,15-20,7,7-10,2.929196,C
1,15.408756,15-20,0,0-3,3.042915,B
2,4.210570,0-5,26,16+,0.112602,F
3,10.028829,10-15,14,11-15,2.054218,D
4,4.672495,0-5,17,16+,1.288061,F


In [3]:
# %%
# Membangun model dengan pendekatan yang lebih aman
def bangun_model(data):
    try:
        # Membuat struktur jaringan
        model = DiscreteBayesianNetwork([
            ('StudyTime_Cat', 'GPA_Cat'),
            ('Absences_Cat', 'GPA_Cat'),
            ('Tutoring', 'GPA_Cat'),
            ('GPA_Cat', 'GradeClass')
        ])
        
        # Membagi data latih dan uji
        train_data, test_data = train_test_split(
            data,
            test_size=0.2,
            random_state=42,
            stratify=data['GradeClass']
        )
        
        # Melatih model dengan penanganan error
        model.fit(
            train_data,
            estimator=MaximumLikelihoodEstimator,
            complete_samples_only=True  # Hanya gunakan sampel lengkap
        )
        
        return model, train_data, test_data
        
    except Exception as e:
        print(f"Error dalam membangun model: {e}")
        return None, None, None

model, train_data, test_data = bangun_model(data_diskrit)

if model is not None:
    print("\nModel berhasil dibangun. Struktur jaringan:")
    for edge in model.edges():
        print(f"{edge[0]} -> {edge[1]}")

INFO:pgmpy: Datatype (N=numerical, C=Categorical Unordered, O=Categorical Ordered) inferred from data: 
 {'StudentID': 'N', 'Age': 'N', 'Gender': 'N', 'Ethnicity': 'N', 'ParentalEducation': 'N', 'StudyTimeWeekly': 'N', 'Absences': 'N', 'Tutoring': 'N', 'ParentalSupport': 'N', 'Extracurricular': 'N', 'Sports': 'N', 'Music': 'N', 'Volunteering': 'N', 'GPA': 'N', 'GradeClass': 'N', 'StudyTime_Cat': 'O', 'Absences_Cat': 'O', 'GPA_Cat': 'O'}


Error dalam membangun model: MaximumLikelihoodEstimator.get_parameters() got an unexpected keyword argument 'complete_samples_only'


In [4]:
# %%
def evaluasi_model_aman(data_uji, model):
    """Fungsi evaluasi dengan penanganan error yang lebih baik"""
    try:
        inferensi = VariableElimination(model)
        prediksi = []
        
        for _, row in data_uji.iterrows():
            try:
                # Pastikan evidence ada dalam data
                evidence = {
                    'StudyTime_Cat': row['StudyTime_Cat'],
                    'Absences_Cat': row['Absences_Cat'],
                    'Tutoring': row['Tutoring']
                }
                
                # Validasi evidence sebelum query
                for var, val in evidence.items():
                    if val not in model.get_cpds(var).state_names[var]:
                        print(f"Peringatan: Nilai {val} tidak ada di variabel {var}")
                        raise ValueError(f"Nilai tidak valid untuk {var}")
                
                hasil = inferensi.map_query(
                    variables=['GradeClass'],
                    evidence=evidence
                )
                prediksi.append(hasil['GradeClass'])
                
            except Exception as e:
                print(f"Error pada baris {_}: {e}")
                prediksi.append(np.nan)  # Gunakan NaN jika error
        
        # Hapus prediksi yang error (NaN)
        valid_idx = ~pd.isna(prediksi)
        prediksi_valid = [prediksi[i] for i in range(len(prediksi)) if valid_idx[i]]
        aktual_valid = data_uji['GradeClass'][valid_idx]
        
        if len(prediksi_valid) > 0:
            akurasi = accuracy_score(aktual_valid, prediksi_valid)
            print(f"\nAkurasi Prediksi: {akurasi:.2f}")
            
            print("\nMatriks Konfusi:")
            print(pd.crosstab(aktual_valid, prediksi_valid,
                            rownames=['Aktual'], colnames=['Prediksi']))
        else:
            print("Tidak ada prediksi valid yang dihasilkan.")
            
    except Exception as e:
        print(f"Error dalam evaluasi model: {e}")

if model is not None:
    evaluasi_model_aman(test_data, model)

In [5]:
# %%
def prediksi_interaktif_aman(waktu_belajar, absensi, bimbingan):
    """Fungsi prediksi dengan penanganan error yang lebih baik"""
    try:
        if model is None:
            print("Model belum dibangun. Mohon jalankan sel sebelumnya.")
            return
            
        # Konversi input ke kategori
        waktu_cat = pd.cut([waktu_belajar], 
                         bins=[0,5,10,15,20,np.inf],
                         labels=['0-5','5-10','10-15','15-20','20+'],
                         right=False)[0]
        
        absen_cat = pd.cut([absensi],
                         bins=[0,3,6,10,15,np.inf],
                         labels=['0-3','4-6','7-10','11-15','16+'],
                         right=False)[0]
        
        # Validasi input
        if waktu_cat not in model.get_cpds('StudyTime_Cat').state_names['StudyTime_Cat']:
            print(f"Peringatan: Waktu belajar {waktu_belajar} tidak valid")
            return
            
        if absen_cat not in model.get_cpds('Absences_Cat').state_names['Absences_Cat']:
            print(f"Peringatan: Absensi {absensi} tidak valid")
            return
            
        if bimbingan not in [0, 1]:
            print("Peringatan: Nilai bimbingan harus 0 atau 1")
            return
        
        # Lakukan prediksi
        inferensi = VariableElimination(model)
        hasil = inferensi.query(
            ['GradeClass'],
            evidence={
                'StudyTime_Cat': waktu_cat,
                'Absences_Cat': absen_cat,
                'Tutoring': bimbingan
            }
        )
        
        # Visualisasi hasil
        plt.figure(figsize=(10,5))
        grade_order = ['A', 'B', 'C', 'D', 'F']
        sns.barplot(x=grade_order, y=[hasil.values[grade_order.index(g)] for g in grade_order])
        plt.title('Probabilitas Prediksi Nilai')
        plt.ylabel('Probabilitas')
        plt.xlabel('Grade')
        plt.show()
        
        # Hasil utama
        grade_pred = grade_order[np.argmax([hasil.values[grade_order.index(g)] for g in grade_order])]
        print(f"\nPrediksi Grade: {grade_pred}")
        print(f"Probabilitas: {np.max(hasil.values):.2f}")
        
    except Exception as e:
        print(f"Error dalam prediksi: {e}")

# Membuat widget interaktif yang aman
if model is not None:
    waktu_slider = widgets.IntSlider(min=0, max=40, value=10, description='Waktu Belajar:')
    absen_slider = widgets.IntSlider(min=0, max=30, value=5, description='Absensi:')
    bimbingan_dropdown = widgets.Dropdown(
        options=[('Tidak', 0), ('Ya', 1)],
        value=0,
        description='Bimbingan:'
    )

    display(widgets.interactive(
        prediksi_interaktif_aman,
        waktu_belajar=waktu_slider,
        absensi=absen_slider,
        bimbingan=bimbingan_dropdown
    ))
else:
    print("Tidak dapat membuat antarmuka karena model tidak tersedia.")

Tidak dapat membuat antarmuka karena model tidak tersedia.


# Three

In [3]:
# %% [markdown]
# ## Solusi Lengkap Prediksi Performa Mahasiswa
# ### Dengan Penanganan Error dan Validasi Model

# %%
# 1. IMPORT LIBRARY
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pgmpy.models import DiscreteBayesianNetwork
from pgmpy.estimators import MaximumLikelihoodEstimator
from pgmpy.inference import VariableElimination
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import ipywidgets as widgets
from IPython.display import display, clear_output
import warnings
warnings.filterwarnings('ignore')

# %% [markdown]
# ## 2. PEMROSESAN DATA DENGAN VALIDASI

# %%
def load_and_preprocess_data():
    """Memuat dan memproses data dengan penanganan error"""
    try:
        # Load data
        data = pd.read_csv('./../Student_performance_data_.csv')
        print("✅ Data berhasil dimuat")
        
        # Validasi kolom penting
        required_cols = ['StudyTimeWeekly', 'Absences', 'Tutoring', 'GPA', 'GradeClass']
        missing_cols = [col for col in required_cols if col not in data.columns]
        
        if missing_cols:
            print(f"❌ Kolom yang hilang: {missing_cols}")
            return None
        
        # Diskritisasi
        data['StudyTime_Cat'] = pd.cut(data['StudyTimeWeekly'],
                                     bins=[0,5,10,15,20,np.inf],
                                     labels=['0-5','5-10','10-15','15-20','20+'])
        
        data['Absences_Cat'] = pd.cut(data['Absences'],
                                    bins=[0,3,6,10,15,np.inf],
                                    labels=['0-3','4-6','7-10','11-15','16+'])
        
        data['GPA_Cat'] = pd.cut(data['GPA'],
                                bins=[0,2.0,2.5,3.0,3.5,4.0],
                                labels=['F','D','C','B','A'])
        
        print("✅ Diskritisasi data berhasil")
        return data
    
    except Exception as e:
        print(f"❌ Error dalam memproses data: {str(e)}")
        return None

# Jalankan fungsi pemrosesan data
processed_data = load_and_preprocess_data()

if processed_data is not None:
    display(processed_data.head())

# %% [markdown]
# ## 3. PEMBANGUNAN MODEL DENGAN PENANGANAN ERROR

# %%
def build_bayesian_network(data):
    """Membangun model Bayesian Network dengan validasi"""
    try:
        # 1. Definisikan struktur jaringan
        model = DiscreteBayesianNetwork([
            ('StudyTime_Cat', 'GPA_Cat'),
            ('Absences_Cat', 'GPA_Cat'), 
            ('Tutoring', 'GPA_Cat'),
            ('GPA_Cat', 'GradeClass')
        ])
        
        # 2. Bagi data training-testing
        train_data, test_data = train_test_split(
            data, 
            test_size=0.2,
            random_state=42,
            stratify=data['GradeClass']
        )
        
        # 3. Latih model
        model.fit(
            train_data,
            estimator=MaximumLikelihoodEstimator,
            complete_samples_only=True
        )
        
        print("✅ Model berhasil dibangun dan dilatih")
        return model, test_data
    
    except Exception as e:
        print(f"❌ Gagal membangun model: {str(e)}")
        return None, None

# Bangun model
bn_model, test_data = build_bayesian_network(processed_data)

# %% [markdown]
# ## 4. EVALUASI MODEL

# %%
def evaluate_model(model, test_data):
    """Evaluasi model dengan penanganan kasus error"""
    if model is None:
        print("❌ Model tidak tersedia untuk evaluasi")
        return
    
    try:
        inference = VariableElimination(model)
        predictions = []
        
        for _, row in test_data.iterrows():
            try:
                # Buat evidence dengan validasi
                evidence = {
                    'StudyTime_Cat': row['StudyTime_Cat'],
                    'Absences_Cat': row['Absences_Cat'],
                    'Tutoring': row['Tutoring']
                }
                
                # Prediksi
                result = inference.map_query(['GradeClass'], evidence=evidence)
                predictions.append(result['GradeClass'])
            except:
                predictions.append(np.nan)
        
        # Hitung akurasi
        valid_preds = [p for p in predictions if not pd.isna(p)]
        if len(valid_preds) > 0:
            accuracy = accuracy_score(
                test_data['GradeClass'][:len(valid_preds)], 
                valid_preds
            )
            print(f"📊 Akurasi Model: {accuracy:.2f}")
        else:
            print("⚠ Tidak ada prediksi valid yang dihasilkan")
            
    except Exception as e:
        print(f"❌ Error dalam evaluasi: {str(e)}")

# Jalankan evaluasi
evaluate_model(bn_model, test_data)

# %% [markdown]
# ## 5. ANTARMUKA INTERAKTIF YANG AMAN

# %%
def create_interactive_interface(model):
    """Membuat antarmuka prediksi interaktif"""
    if model is None:
        print("❌ Model tidak tersedia - tidak dapat membuat antarmuka")
        return None
    
    # Style untuk widget
    style = {'description_width': '150px'}
    layout = widgets.Layout(width='400px')
    
    # Widget input
    study_time = widgets.IntSlider(
        min=0, max=40, value=10,
        description='Waktu Belajar (jam/minggu):',
        style=style, layout=layout
    )
    
    absences = widgets.IntSlider(
        min=0, max=30, value=5,
        description='Jumlah Absensi:',
        style=style, layout=layout
    )
    
    tutoring = widgets.Dropdown(
        options=[('Tidak', 0), ('Ya', 1)],
        value=0,
        description='Mengikuti Bimbingan:',
        style=style, layout=layout
    )
    
    # Output widget
    output = widgets.Output()
    
    # Fungsi prediksi
    def predict(study, absent, tutor):
        with output:
            clear_output(wait=True)
            try:
                # Konversi ke kategori
                study_cat = pd.cut([study], [0,5,10,15,20,np.inf],
                                 labels=['0-5','5-10','10-15','15-20','20+'])[0]
                absent_cat = pd.cut([absent], [0,3,6,10,15,np.inf],
                                  labels=['0-3','4-6','7-10','11-15','16+'])[0]
                
                # Prediksi
                inference = VariableElimination(model)
                result = inference.query(
                    ['GradeClass'],
                    evidence={
                        'StudyTime_Cat': study_cat,
                        'Absences_Cat': absent_cat,
                        'Tutoring': tutor
                    }
                )
                
                # Visualisasi
                plt.figure(figsize=(10,5))
                grades = ['A', 'B', 'C', 'D', 'F']
                probs = [result.values[grades.index(g)] for g in grades]
                
                colors = ['green', 'lightgreen', 'gold', 'orange', 'red']
                bars = plt.bar(grades, probs, color=colors)
                
                # Tambah label probabilitas
                for bar in bars:
                    height = bar.get_height()
                    plt.text(bar.get_x() + bar.get_width()/2., height,
                            f'{height:.2f}', ha='center', va='bottom')
                
                plt.title('Probabilitas Prediksi Nilai')
                plt.ylabel('Probabilitas')
                plt.ylim(0, 1)
                plt.show()
                
                # Hasil utama
                pred_grade = grades[np.argmax(probs)]
                print(f"🎯 Prediksi Nilai: {pred_grade}")
                print(f"🔢 Probabilitas: {max(probs):.2f}")
                
                # Rekomendasi
                print("\n💡 Rekomendasi:")
                if pred_grade in ['D', 'F']:
                    print("- Tingkatkan waktu belajar minimal 10 jam/minggu")
                    print("- Kurangi absensi di bawah 5 kali")
                    if tutor == 0:
                        print("- Pertimbangkan mengikuti bimbingan belajar")
                elif pred_grade == 'C':
                    print("- Pertahankan waktu belajar 10-15 jam/minggu")
                    print("- Ikuti kegiatan ekstrakurikuler")
                else:
                    print("- Pertahankan prestasi Anda!")
                    
            except Exception as e:
                print(f"❌ Error dalam prediksi: {str(e)}")
    
    # Tampilkan antarmuka
    ui = widgets.VBox([
        widgets.HTML("<h2>Prediksi Performa Akademik</h2>"),
        study_time,
        absences, 
        tutoring,
        output
    ])
    
    widgets.interactive_output(predict, {
        'study': study_time,
        'absent': absences,
        'tutor': tutoring
    })
    
    return ui

# Buat dan tampilkan antarmuka
interface = create_interactive_interface(bn_model)
if interface is not None:
    display(interface)
else:
    print("Silakan perbaiki model terlebih dahulu sebelum membuat antarmuka")

✅ Data berhasil dimuat
✅ Diskritisasi data berhasil


,StudentID,Age,Gender,Ethnicity,ParentalEducation,StudyTimeWeekly,Absences,Tutoring,ParentalSupport,Extracurricular,Sports,Music,Volunteering,GPA,GradeClass,StudyTime_Cat,Absences_Cat,GPA_Cat
0,1001,17,1,0,2,19.833723,7,1,2,0,0,1,0,2.929196,2.0,15-20,7-10,C
1,1002,18,0,0,1,15.408756,0,0,1,0,0,0,0,3.042915,1.0,15-20,NaN,B
2,1003,15,0,2,3,4.210570,26,0,2,0,0,0,0,0.112602,4.0,0-5,16+,F
3,1004,17,1,0,3,10.028829,14,0,3,1,0,0,0,2.054218,3.0,10-15,11-15,D
4,1005,17,1,0,2,4.672495,17,1,3,0,0,0,0,1.288061,4.0,0-5,16+,F


INFO:pgmpy: Datatype (N=numerical, C=Categorical Unordered, O=Categorical Ordered) inferred from data: 
 {'StudentID': 'N', 'Age': 'N', 'Gender': 'N', 'Ethnicity': 'N', 'ParentalEducation': 'N', 'StudyTimeWeekly': 'N', 'Absences': 'N', 'Tutoring': 'N', 'ParentalSupport': 'N', 'Extracurricular': 'N', 'Sports': 'N', 'Music': 'N', 'Volunteering': 'N', 'GPA': 'N', 'GradeClass': 'N', 'StudyTime_Cat': 'O', 'Absences_Cat': 'O', 'GPA_Cat': 'O'}


❌ Gagal membangun model: MaximumLikelihoodEstimator.get_parameters() got an unexpected keyword argument 'complete_samples_only'
❌ Model tidak tersedia untuk evaluasi
❌ Model tidak tersedia - tidak dapat membuat antarmuka
Silakan perbaiki model terlebih dahulu sebelum membuat antarmuka
